## Technical indicators tester

pandas-ta provides more than to 200 indicators, so we dont have to do the calculations:

https://github.com/twopirllc/pandas-ta

#### help with ta
df = pd.DataFrame()

#### Help about this, 'ta', extension
help(df.ta)

#### List of all indicators
df.ta.indicators()

#### Help about an indicator such as bbands
help(ta.sma)

In [1]:
from binance_api import BinanceAPI
api = BinanceAPI(
    verbose=True,
    pair="BTCUSDT",
    difficulty="MEDIUM",
    use_fee=True,
    fee_maker=0.0002,
    fee_taker=0.0004,
    system="NETTING"
)
def load_data():
    start = ["2023-10-01", "2023-08-01", "2023-12-10", "2023-11-01", "2024-02-03 23:47:00", "2019-09-02"]
    end = ["2023-11-19", "2023-09-01", "2024-01-10", "2024-01-30", "2024-02-04 05:01:00", "2022-12-31"]
    num = 0
    data = api.load_data(
        interval_of_candles="1h",
        start_date_utc=start[num],
        end_date_utc=end[num],
    )
    return data
# api.calculate_hold_strategy(initial_quote=100)
#data

In [2]:
import pandas as pd
from orders import Position
from typing import Union
from datetime import datetime

### Bollinger Bands

In [4]:
data = load_data() # resets data
import pandas as pd
from orders import Position
from typing import Union
from datetime import datetime


class BollingerBands():
    def __init__(
        self,
        data: pd.DataFrame,
        column: str = "Close",
        prefix: str = "price",
        dev: float = 1.0,
        periods: int = 50,
    ) -> None:
        """
        Inits attributes of class.
        data: dataframe with info
        column: column used to calculate strategy
        prefix: prefix used to store cols
        dev: standard deviation used
        periods: periods used for calculations.
        """
        self.data = data
        self.column = column
        self.prefix = prefix
        self.dev = dev
        self.periods = periods
        self.BBS = prefix + "|BBs|" + str(dev) + "|" + str(periods)
        self.BBS_distance = self.BBS + "|Distance"
        self.BBS_upper = self.BBS + "|Upper"
        self.BBS_lower = self.BBS + "|Lower"
        self.SMA = prefix + "|SMA|" + str(periods)
        self.last_position = Position.NEUTRAL

    def enough_info_to_predict(
        self,
        index: Union[datetime, pd.Timestamp]
    ) -> bool:
        """
        Tells if position can be predicted
        """
        idx_num = self.data.index.get_loc(index)
        return idx_num + 1 >= self.periods

    def calculate(self, force: bool = False) -> None:
        """Calculates strategy for all dataframe"""
        if self.BBS_distance in self.data.columns and not force:
            return

        if self.SMA not in self.data.columns or force:
            SM = self.data[self.column].rolling(self.periods)
            self.data[self.SMA] = SM.mean()

        std_dev = SM.std()
        self.data[self.BBS_lower] = self.data[self.SMA] - std_dev * self.dev
        self.data[self.BBS_upper] = self.data[self.SMA] + std_dev * self.dev
        self.data[
            self.BBS_distance
        ] = self.data[self.column] - self.data[self.SMA]

    def calculate_for_row(
        self,
        index: Union[datetime, pd.Timestamp]
    ) -> None:
        """Calculate strategy for some row using index"""
        index_num = self.data.index.get_loc(index)
        SM = self.data[self.column][
            index_num+1-self.periods:index_num+1
        ].rolling(self.periods)
        std_dev = SM.std().iloc[-1]
        sma = SM.mean().iloc[-1]

        self.data.loc[index, self.SMA] = sma
        self.data.loc[index, self.BBS_lower] = sma - std_dev * self.dev
        self.data.loc[index, self.BBS_upper] = sma + std_dev * self.dev
        self.data.loc[
            index, self.BBS_distance
        ] = self.data.loc[index, self.column] - sma

    def strategy(
        self,
        index: Union[datetime, pd.Timestamp]
    ) -> Position:
        '''Returns predicted position'''
        if not self.enough_info_to_predict(index=index):
            self.last_position = Position.NEUTRAL
            return self.last_position
        idx_num = self.data.index.get_loc(index)

        curr_value = self.data.loc[index, self.column]
        prev_distance = self.data[self.BBS_distance][idx_num-1]
        distance = self.data[self.BBS_distance][idx_num]

        if curr_value < self.data.loc[index, self.BBS_lower]:
            self.last_position = Position.LONG
        elif curr_value > self.data.loc[index, self.BBS_upper]:
            self.last_position = Position.SHORT
        elif distance * prev_distance < 0:
            self.last_position = Position.NEUTRAL
        return self.last_position
    
bbs = BollingerBands(
    data = data,
    dev = 1, 
    periods = 50,
    column = "Close",
)
#bbs.calculate() #calculate for all dataframe
bbs.calculate_for_row(index= data.index[-1]) #calculate just for last row
print(bbs.strategy(index=data.index[-1])) #print strategy for last row
data

Trying to load info from directory...
Data loaded.
Position.NEUTRAL


/tmp/ipykernel_87172/1094911102.py:88: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif self.data[self.BBS_distance][idx_num] * self.data[self.BBS_distance][idx_num-1] < 0:


,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,price|SMA|50,price|BBs|1|50|Lower,price|BBs|1|50|Upper,price|BBs|1|50|Distance
Date,,,,,,,,,,,,,
2023-10-01 00:00:00,26951.0,26986.6,26939.0,26976.9,2733.691,7.370182e+07,36206,1615.927,4.357174e+07,NaN,NaN,NaN,NaN
2023-10-01 01:00:00,26976.9,27026.8,26963.8,27005.1,3161.822,8.536434e+07,38429,1787.589,4.826482e+07,NaN,NaN,NaN,NaN
2023-10-01 02:00:00,27005.1,27037.5,27004.4,27015.6,2681.754,7.246509e+07,35867,1405.772,3.798779e+07,NaN,NaN,NaN,NaN
2023-10-01 03:00:00,27015.7,27039.7,26993.6,27033.0,2415.426,6.526980e+07,28271,1086.230,2.935505e+07,NaN,NaN,NaN,NaN
2023-10-01 04:00:00,27032.9,27035.0,27004.2,27035.0,1889.918,5.106637e+07,25342,804.897,2.175002e+07,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-18 20:00:00,36688.4,36720.0,36537.8,36604.1,7269.000,2.660983e+08,76916,2719.743,9.955807e+07,NaN,NaN,NaN,NaN
2023-11-18 21:00:00,36604.0,36626.0,36530.7,36550.0,2961.721,1.083138e+08,44733,1353.136,4.948897e+07,NaN,NaN,NaN,NaN
2023-11-18 22:00:00,36550.0,36569.9,36450.0,36517.4,4182.515,1.526673e+08,56610,1798.456,6.565008e+07,NaN,NaN,NaN,NaN
